# 0. Install Dependencies

In [4]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.3.0


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827644 sha256=2f8dc86d41addd39bcbe0a852f7c63e598464d201e63a436d040418acc14f370
  Stored in directory: c:\users\kamalesh dran\appdata\local\pip\cache\wheels\af\2b\30\5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym


# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class RoomEnv(Env):
    def __init__(self):
        # Actions we can take - down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low = np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 28 + random.randint(-3,3)
        # Set AC length
        self.ac_length = 60
                                     
    def step(self, action):
        # Apply action
        self.state += action -1
        # Reduce AC length by 1 second
        self.ac_length -= 1
                                     
        #Calculate reward                             
        if self.state >=27 and self.state <=29:
            reward = 1
        else:
            reward = -1
                                     
        #Check if AC is done                             
        if self.ac_length <=0:
            done = True
        else: 
            done = False
                                     
        # Apply temperature noise 
        self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
                                     
        # Return step information
        return self.state, reward, done, info
                                     
    def reset(self):
        # Reset room temperature
        self.state = 28 + random.randint(-3,3)
        # Reset AC time
        self.ac_length = 60
        return self.state

In [4]:
env = RoomEnv()

C:\Users\KAMALESH DRAN\anaconda3\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
env.action_space.sample()

0

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-52
Episode:2 Score:-60
Episode:3 Score:-60
Episode:4 Score:-46
Episode:5 Score:-60
Episode:6 Score:-24
Episode:7 Score:-46
Episode:8 Score:-60
Episode:9 Score:-60
Episode:10 Score:-50


# 2. Create a Deep Learning Model with Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
actions

3

In [10]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [20]:
del model

In [21]:
model = build_model(states, actions)

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                48        
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [17]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [18]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [23]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 14:22 - reward: -1.0000

C:\Users\KAMALESH DRAN\anaconda3\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\KAMALESH DRAN\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 66s 7ms/step - reward: -0.8266
166 episodes - episode_reward: -49.602 [-60.000, 2.000] - loss: 1.844 - mae: 9.298 - mean_q: -12.791

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: -0.7718
167 episodes - episode_reward: -46.287 [-60.000, -4.000] - loss: 2.156 - mae: 11.627 - mean_q: -16.461

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.6004
167 episodes - episode_reward: -36.072 [-60.000, 10.000] - loss: 1.909 - mae: 10.588 - mean_q: -14.889

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: -0.2132
166 episodes - episode_reward: -12.795 [-60.000, 28.000] - loss: 1.005 - mae: 6.272 - mean_q: -8.281

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: 0.0144
done, took 342.745 seconds


In [24]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 52.000, steps: 60
Episode 2: reward: 38.000, steps: 60
Episode 3: reward: 32.000, steps: 60
Episode 4: reward: 52.000, steps: 60
Episode 5: reward: 44.000, steps: 60
Episode 6: reward: 44.000, steps: 60
Episode 7: reward: 46.000, steps: 60
Episode 8: reward: 46.000, steps: 60
Episode 9: reward: 30.000, steps: 60
Episode 10: reward: 46.000, steps: 60
Episode 11: reward: 42.000, steps: 60
Episode 12: reward: 46.000, steps: 60
Episode 13: reward: 44.000, steps: 60
Episode 14: reward: 34.000, steps: 60
Episode 15: reward: 48.000, steps: 60
Episode 16: reward: 50.000, steps: 60
Episode 17: reward: 52.000, steps: 60
Episode 18: reward: 42.000, steps: 60
Episode 19: reward: 46.000, steps: 60
Episode 20: reward: 44.000, steps: 60
Episode 21: reward: 46.000, steps: 60
Episode 22: reward: 42.000, steps: 60
Episode 23: reward: 40.000, steps: 60
Episode 24: reward: 48.000, steps: 60
Episode 25: reward: 36.000, steps: 60
Episode 26: reward: 52.000, st